In [1]:
import importlib
# --------------------
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
import numpy as np
from numpy.linalg import norm,inv,matrix_rank

import h5py
from sklearn.cross_decomposition import PLSRegression

import warnings
warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [2]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)

    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )

def Comp_Model_Score(estimator, cv, X, Y):
    score = np.zeros((n_fold,))
    for i,(train_index, test_index) in enumerate(cv):
        try:
            estimator.fit(X[train_index], Y[train_index])
            y_true = Y[test_index]
            y_pred = estimator.predict(X[test_index])
            score[i] = RMSE(y_true, y_pred)
        except:
            score[i] = np.nan
            break
    return np.mean(score)

In [3]:
with h5py.File('./data/merra2_t.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)
        
    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
 

4412


In [4]:
print(X_train.shape)
print(Y_train.shape)
n_fold = 74
test_size=30
tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break
    
print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break    

(4412, 30456)
(4412, 30456)
Fold 0:
  Train: len=2192
  Test:  len=30
Fold 1:
  Train: len=2222
  Test:  len=30
Fold 2:
  Train: len=2252
  Test:  len=30
Fold 3:
  Train: len=2282
  Test:  len=30
Fold 4:
  Train: len=2312
  Test:  len=30
  Train: [   0    1    2 ... 2309 2310 2311]
  Test:  [2312 2313 2314 2315 2316 2317 2318 2319 2320 2321 2322 2323 2324 2325
 2326 2327 2328 2329 2330 2331 2332 2333 2334 2335 2336 2337 2338 2339
 2340 2341]
Fold 0:
  Train: len=2192
  Test:  len=30
Fold 1:
  Train: len=30
  Test:  len=30
Fold 2:
  Train: len=30
  Test:  len=30
Fold 3:
  Train: len=30
  Test:  len=30
Fold 4:
  Train: len=30
  Test:  len=30
  Train: [2282 2283 2284 2285 2286 2287 2288 2289 2290 2291 2292 2293 2294 2295
 2296 2297 2298 2299 2300 2301 2302 2303 2304 2305 2306 2307 2308 2309
 2310 2311]
  Test:  [2312 2313 2314 2315 2316 2317 2318 2319 2320 2321 2322 2323 2324 2325
 2326 2327 2328 2329 2330 2331 2332 2333 2334 2335 2336 2337 2338 2339
 2340 2341]


In [ ]:
import Code.SIMPLS
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

n_comp_lst=list(range(1,16))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(SIMPLS(),  parameters,
                   cv=tscv, n_jobs=4, verbose=3,  
                   scoring='neg_root_mean_squared_error')
print(clf)
tim.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tim.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))


In [ ]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

scores = []
params = []
n_comp_list=list(range(100,401,50))

tim_tot.tic()
for n_comp in n_comp_list:
    tim.tic()
    pls=ISIMPLS(n_components=n_comp)
    scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
    params.append({'n_components': n_comp})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")


params={'n_components': 100}, score=2.1730468e+00, elapsed time=97598.1s


params={'n_components': 150}, score=2.2380231e+00, elapsed time=159673.6s


In [5]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

scores = []
params = []
n_comp_list=list(range(200,401,50))

tim_tot.tic()
for n_comp in n_comp_list:
    tim.tic()
    pls=ISIMPLS(n_components=n_comp)
    scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
    params.append({'n_components': n_comp})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")

params={'n_components': 200}, score=nan, elapsed time=91419.3s


params={'n_components': 250}, score=nan, elapsed time=8.5s


params={'n_components': 300}, score=nan, elapsed time=8.8s


params={'n_components': 350}, score=nan, elapsed time=8.4s


params={'n_components': 400}, score=nan, elapsed time=0.3s


ValueError: All-NaN slice encountered